Noah Crowley

nwc17

## Overview

For this project, I will be trying to go a step beyond anomaly detection. I will be hoping to detect anomalies in time series data using the theoretical example of running a social media website, but more importantly I will be trying to use Bayesian networks to find the most likely cause of the anomaly. In addition, I plan to create a proof-of-concept for having a confirmation system to help the network improve its weights and other variables.

## Anomaly Detection

### Explanation

For the anomaly detection, I will be using the normalized (in the range $[-1, \infty)$) slope between time t-1 and t, where t is the time of the most recent reading and t-1 is the time of the reading just prior, where each reading is taken once every R seconds (R stands for resolution). I will compare the $N$ most recent slope readings with a set of normal distributions that have the historical mean slope for those times ($\mu_t$) and the historical variances in slope for those times ($\sigma_t^2$) as parameters. An anomaly will be reported if the combined probability of these slopes drops below a certain threshold ($M$). That is there is an anomaly if:

$$
\begin{align}
ln(M) & \geq \sum_{j=0}^{N - 1} ln(P(r_{t-j})) \\
& = \sum_{j=0}^{N - 1} \bigg( ln\big(\dfrac{1}{\sigma_{t-j} \sqrt{2\pi}} \exp({\dfrac{-(r_{t-j} - \mu_{t-j})^2}{2\sigma^2}})\big) \bigg) \\
& = \sum_{j=0}^{N - 1} \bigg( ln\big(\dfrac{1}{\sigma_{t-j} \sqrt{2\pi}}\big) + \dfrac{-(r_{t-j} - \mu_{t-j})^2}{2\sigma^2} \bigg) \\
& = \sum_{j=0}^{N - 1} \bigg( \dfrac{-(r_{t-j} - \mu_{t-j})^2}{2\sigma^2} - ln(\sigma_{t-j}) - ln(\sqrt{2\pi}) \bigg) \\
& = \sum_{j=0}^{N - 1} \bigg( \dfrac{-(r_{t-j} - \mu_{t-j})^2}{2\sigma^2} - ln(\sigma_{t-j}) \bigg) - 3ln(\sqrt{2\pi})
\end{align}
$$

### Implementation

The first step is to have the data of the means and variances. While this could use actual historical data gathered from a database, I will just generate a sinusoidal pattern, giving me a mean for every second of the day (86400 data points). I will use the same variance across the board - $(0.25)^2 = 0.0625$.

To do this, I will create a function for getting the mean and another for getting the variance. My "t" variable will represent the number of seconds since midnight, and it will be an integer. I will also create a function for wrapping "t" so that it is always in the range [0, 86399].

In [1]:
import math

MAX_T = 86400

def _wrap_time(t):
    return t % MAX_T

def _get_mean_at_time(t):
    t = _wrap_time(t)
    mean = math.sin(2 * math.pi * (t / MAX_T))
    return mean
    
def _get_variance_at_time(t):
    return 0.0625

Since this is not using actual data, I will also create a function to generate synthetic data.

In [2]:
import numpy as np

def generate_synthetic_data():
    data = [0] * MAX_T
    for t in range(MAX_T):
        mu = get_mean_at_time(t)
        sigma = np.sqrt(get_variance_at_time(t))
        data[t] = np.random.normal(mu, sigma)
    return data

And now I can actually create the function for detecting anomalies. It will take an array of values, a current time, a threshold, as well as two functions with which to get the mean and variance for the time $t$. It will then simply return True or False.

In [3]:
def test_for_anomaly(recent_data, t, threshold, get_mean_at_time, get_variance_at_time):
    n = len(recent_data)
    log_probability = (-1) * n * np.log(np.sqrt(2*math.pi))
    
    for j in range(n):
        r = recent_data[n - 1 - j]
        mu = get_mean_at_time(t - j)
        variance = get_variance_at_time(t)
        sigma = np.sqrt(variance)
        
        log_probability += (-1)*((r - mu)**2 / (2 * variance)) - np.log(sigma)
        
    probability = np.exp(log_probability)
        
    return threshold >= probability

## Source Identification

### Explanation

For the source identification, I will create a Bayesian belief network where my evidence variables are whether there is an anomaly detected in a given service (as described above) and when the most recent changes were published for each service. My hidden variables, the values I am trying to predict, are then just whether a given service is responsible for an anomaly. Services will have edges connecting them to all time series they directly impact and their own respective last-changed node. The edges will have variable weights, essentially affecting the likelihood that each service has of being the cause of an anomaly for each individual time series. As will be explained later, the idea is that these weights will be capable of being updated by a user confirming or denying whether the prediction turned out to be accurate.

### Implementation

#### Priors

To implement this source identification, I will need to decide upon distributions for my three types of nodes. My anomaly detection nodes have already been figured out above. This leaves me with the individual service failures and the recent update nodes.

It seems fitting for the individual service failures to have a simple binomial distribution with a low prior probability of failure (as systems are assumed to be operating properly the vast majority of the time). In a real application, this data could be computed from historical data, but for the sake of this demonstration I will assume the prior to be:

| System status | Probability |
| --- | --- |
| System operating properly | $0.9990$ |
| System failure | $0.0010$ |

As I'll be using the pgmpy package and its Bayesian Model class does not allow for continuous variables, it will be easier to break the recent update nodes into a discrete set of values. These values will be:

 - &le; 1min since last update
 - &le; 5min since last update
 - &le; 15min since last update
 - &gt; 15min since last update
 
To derive probabilities for these, I could again use the historical data to find the average number of updates a service receives per day, but that would still end up just being a scalar multiple of the values I will assume. For now, I will assume every service receives an average of one update per day, meaning the following distribution table would follow:

| Time since last update | Probability |
| --- | --- |
| &le; 1min |  $0.0007$ |
| &le; 5min |  $0.0035$ |
| &le; 15min | $0.0104$ |
| &gt; 15min | $0.9854$ |

I will still need a table of priors for my anomaly detection nodes. For those, I can just use the exponential value of the threshold&mdash;$e^M$&mdash;since the threshold is set for the log probability of the combined $N$ previous slope readings. I'll put this into table form:

| Anomaly status | Probability |
| --- | --- |
| None detected | $1 - e^M$ |
| Anomaly detected | $e^M$ |

#### Conditional Probability Distributions

In order to derive conditional probability distributions, I will need to account for the fact that my model is meant to take in any number of connections to one service's usage graph. Since the latest update nodes have no parents in the model, their CPDs are exactly the same as the original distribution, simplifying the model quite a bit. The service failure nodes are also fairly simple, as their parents are just their most recent update. So while this is the key node set that I will be trying to predict, its actual CPD is simple. The difficult CPD is the final layer&mdash;the anomaly detection nodes. These nodes can be connected to any number of service failures, causing cascading failures throughout a variety of timeseries all at once. So there is an $L \times K$ binary matrix of possible connections between services and graphs, where $L$ is the number of services and $K$ is the number of timeseries.

For example, imagine a website like Facebook, and for a moment imagine that the user session service has been updated. If this update causes the website to be unable to verify a user's identity, the entire website may being to send `HTTP 403 Forbidden` codes to every user, which would likely cause an immediate drop in the number of users online, the number of chat messages being sent, and just about every other timeseries connected to user activity. Meanwhile, if another update were made, such as perhaps an update that slowed down the delivery of chat messages substantially, the number of users online may not take a large dip but the number of messages sent per second would almost certainly fall.

So, what are these CPDs?

As stated before, the update nodes retain their priors as their entire CPD, so that does not change. But the service failure nodes will now have a slightly more complicated distribution. This CPD could again be computed given historical data, but for the sake of demonstration I will create a set of values myself. Let T be the time since the last update:

| System status | T &le; 1min | T &le; 5min | T &le; 15min | T &gt; 15min |
| --- | --- | --- | --- | --- |
| System operating properly | 0.8725 | 0.9350 | 0.9880 | 0.9999 |
| System failure | 0.1275 | 0.0650 | 0.0120 | 0.0001 |

The anomaly nodes are not so easily described in a table. Rather, we will need to derive a formula for this CPD.

Let $a$ be a binary value where 1 represents an anomaly was detected and 0 otherwise, and let $\bar{a}$ be the opposite of $a$. Let $\vec{s} = [s_1, ..., s_d]^T$ represent the state of all $d$ connected services to this timeseries, with each $s_i$ being a binary variable with 1 representing a failure and 0 otherwise. The probability $p(a \mid \vec{s})$ is then:

$$
p(a \mid \vec{s}) = \dfrac{p(\vec{s} \mid a)p(a)}{p(\vec{s} \mid a)p(a) + p(\vec{s} \mid \bar{a})p(\bar{a})}
$$

So then, $p(\vec{s} \mid a)$ is now a necessary component.

$$
\begin{align}
p(\vec{s} \mid a) & \propto \prod_{i = 1}^{d} p(a \mid s_i)p(s_i) \\
& \propto \exp\bigg\{\sum_{i = 1}^{d} log(p(a \mid s_i)) + log(p(s_i)) \bigg\}
\end{align}
$$

So we now are able to use log probabilities of simple conditions. In order to calculate the exact value of $p(\vec{s} \mid a)$, I will need to calcualte the probabilities of all possible values of $\vec{s}$. Since the values are all binary, this is a space of $2^d$. In order to get $p(a \mid \vec{s})$ I will also need to calculate for $\bar{a}$. This means I will need a total of $2^{d+1}$ calculations. For small values of $d$, this is not a big deal. However, even with just seven connected services, we quickly see that number jump into the hundreds, and that only gets worse. The good thing, though, is that this operation only takes place once to set up the model, and then all of the values are saved into the model and accessed quickly.

So now all that is left to do is to actually implement a function to generate this model. The function will take:

- avg_updates_per_day - An $L \times 1$ matrix representing the number of times per day each service is updated on average
- svc_failure_cpd_matrix - An $L \times 4$ matrix representing the CPD for each system given the value of its recent update node. This should represent the probability of the service being in a failure state given the state of the update node.
- svc_failure_priors - An $L \times 1$ matrix representing the priors of the probability of each service having a failure, regardless of its most recent update.
- likelihoods - An $L \times K$ matrix representing $p(a_j \mid s_i = 1)$, where $j$ is the number of the timeseries and $i$ is the number of the service. This should be a sparse matrix where 0 represents no causal effect of service $i$ on timeseries $j$. It is assumed that if a service is functioning properly, an anomaly is not going to be caused, as otherwise it seems that the service is not acting correctly.

And it will produce a pgmpy Bayesian Model object of the system.

The first step for getting this function to work is to generate the raw Bayesian model connections.

In [4]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD as tcpd

In [5]:
def create_raw_bayesian_model(likelihoods):
    edges = []
    
    for i in range(likelihoods.shape[0]):
        update_node = "U" + str(i)
        service_node = "S" + str(i)
        edges.append((update_node, service_node))
        for j in range(likelihoods.shape[1]):
            if likelihoods[i][j] > 0:
                edges.append((service_node, "A" + str(j)))
                
    return BayesianModel(edges)

After that, I need to add the CPDs for the update nodes.

In [6]:
# These values were calculated above
base_update_node_cpds = np.asarray([0.0007, 0.0035, 0.0104, 0.9854])

def add_update_node_cpds(avg_updates_per_day, model):
    cpds = []
    
    for i in range(avg_updates_per_day.shape[0]):
        cpds.append(tcpd(
            variable = "U" + str(i), variable_card = 4, values  = [base_update_node_cpds * avg_updates_per_day[i][0]]
        ))
        
    model.add_cpds(*cpds)

And then another function, this time adding the CPDs for the service nodes given the state of their respective update nodes.

In [7]:
def add_service_node_cpds(svc_failure_cpd_matrix, model):
    cpds = []
    
    for i in range(svc_failure_cpd_matrix.shape[0]):
        cpds.append(tcpd(
            variable = "S" + str(i), variable_card = 2, 
            evidence = ["U" + str(i)], evidence_card = [4],
            values  = [
                1 - svc_failure_cpd_matrix[i],
                svc_failure_cpd_matrix[i]
            ]
        ))
        
    model.add_cpds(*cpds)

And now, finally, the addition of the CPDs for the anomaly detection nodes.

In [8]:
def create_anomaly_node_cpd(svc_failure_priors, likelihoods_column, j):
    connected_service_ids = []
    d = 0

    evidence = []
    evidence_card = []

    for i in range(likelihoods_column.shape[0]):
        if likelihoods_column[i] > 0:
            evidence.append("S" + str(i))
            evidence_card.append(2)
            connected_service_ids.append(i)
            d += 1

    cpd_values = [[],[]]

    for permut_id in range(pow(2, d)):
        log_probs = [0, 0] # For both a = 0 and a = 1
        for conn_svc_id in range(d):
            i = connected_service_ids[conn_svc_id]
            if permut_id & pow(2, i) > 0: # If service i is on
                log_probs[0] += np.log(1 - likelihoods_column[i]) + np.log(svc_failure_priors[j][0]) 
                log_probs[1] += np.log(likelihoods_column[i]) + np.log(svc_failure_priors[j][0])

        probs = np.exp(log_probs)
        probs_sum = np.sum(probs)
        probs /= probs_sum
        probs = probs.tolist()

        cpd_values[0].append(probs[0])
        cpd_values[1].append(probs[1])

    return tcpd(
        variable = "A" + str(j), variable_card = 2, 
        evidence = evidence, evidence_card = evidence_card,
        values  = cpd_values
    )

def add_anomaly_node_cpds(svc_failure_priors, likelihoods, model):
    cpds = []
    
    for j in range(likelihoods.shape[1]):
        cpds.append(create_anomaly_node_cpd(svc_failure_priors, likelihoods[:, j], j))
        
    model.add_cpds(*cpds)

I'll put that all together.

In [9]:
def generate_bayesian_model(avg_updates_per_day, svc_failure_cpd_matrix, svc_failure_priors, likelihoods):
    model = create_raw_bayesian_model(likelihoods)
    add_update_node_cpds(avg_updates_per_day, model)
    add_service_node_cpds(svc_failure_cpd_matrix, model)
    add_anomaly_node_cpds(svc_failure_priors, likelihoods, model)
    
    return model

## Improving Beliefs

### Explanation

If this model detects an anomaly, it is either correct or it is not. Updating this is as simple as decreasing the threshold size. Similarly, if the model fails to detect an anomaly, it is as simple as increasing the threshold for the timeseries node(s) that had the anomaly.

The more complicated update is in regards to when the model fails to predict the correct _cause_ of the anomaly. However, even this can be simple&mdash;just slightly increase and decrease values in one column of the likelihoods matrix.

### Implementation

The implementation of these ideas are in the next section.

## Creating the Entire Model

Now I have the two key components ready&mdash;the anomaly detection functions and the Bayesian network creation. But in order to actually be useful, I need to combine these into one object. For this, I will create a class that accepts all the same parameters into its constructor as the `generate_bayesian_model` function as well as a $K \times 1$ matrix of thresholds for anomaly detection, and also two $K \times 86400$ matrices representing the means and variances for each timeseries. I will call this class `TimeseriesNetworkModel`. This model will contain the following functions:

- `record_slope_reading(timeseries_id, reading_value)`  
  This function will save a new timeseries slope reading to its data.  
  
- `record_service_update(service_id)`  
  This function saves the time for the service as its most recent update time.  
  
- `check_for_anomalies(n)`  
  This function runs through each timeseries and checks the most recent $n$ slope readings. This function will return a $K \times 1$ matrix with 1s representing anomalies and 0s otherwise.  
  
- `find_most_likely_source(anomalies_results)`  
  This function returns the service ID of the service it deems most likely for the reading of anomalies. It will return -1 if the anomalies results list says there are no anomalies.
  
- `report_invalid_anomaly_result(timeseries_id, anomaly_was_missed, step_size = 0.001)`  
  This function will change the threshold value related with a particular timeseries for the sake of improving the model. It takes a binary value that says whether an anomaly was missed&mdash;if `True`, then the function will increase the threshold, and if `False` then the model will decrease the threshold. The `step_size` argument determines how much the model will change the threshold.
  
- `report_incorrect_source_estimation(anomalies_results, incorrectly_estimated_service_id, correct_service_id, step_size = 0.001)`  
  This function will vary the CPDs of the Bayesian network to attempt to improve the model. The `step_size` argument determines the magnitude of the changes in the model.

In [10]:
import time
from pgmpy.inference import VariableElimination as VE

class TimeseriesNetworkModel:
    def __init__ (
        self, 
        avg_updates_per_day, svc_failure_cpd_matrix, svc_failure_priors, likelihoods,
        thresholds, means, variances
    ):
        self.avg_updates_per_day = avg_updates_per_day
        self.svc_failure_cpd_matrix = svc_failure_cpd_matrix
        self.svc_failure_priors = svc_failure_priors
        self.likelihoods = likelihoods
        self.thresholds = thresholds
        self.means = means
        self.variances = variances
        
        self.num_services = svc_failure_priors.shape[0]
        self.num_timeseries = thresholds.shape[0]
        
        self.network_bayes_model = generate_bayesian_model(
            avg_updates_per_day, svc_failure_cpd_matrix, svc_failure_priors, likelihoods
        )
        if not self.network_bayes_model.check_model():
            raise Exception("Bayesian model failure")
            
        self.ve = VE(self.network_bayes_model)
        
        self.timeseries_values = [[] for i in range(self.thresholds.shape[0])]
        self.recent_update_times = np.zeros_like(svc_failure_priors)
        
    def record_slope_reading(self, timeseries_id, reading_value):
        self.timeseries_values[timeseries_id].append(reading_value)
        
    def record_service_update(self, service_id):
        self.recent_update_times[service_id][0] = time.time()
        
    def check_for_anomalies(self, n):
        result = np.zeros_like(self.thresholds)
        
        for j in range(self.num_timeseries):
            t = len(self.timeseries_values[j])
            # If there are not yet enough timeseries readings to make a calculation, just go over this
            if t < n:
                continue
            
            recent_values = self.timeseries_values[j][-n:]
            threshold = self.thresholds[j]
            get_mean_at_time = lambda t: self.means[j][_wrap_time(t)]
            get_variance_at_time = lambda t: self.variances[j][_wrap_time(t)]
            
            if test_for_anomaly(recent_values, t, threshold, get_mean_at_time, get_variance_at_time):
                result[j] = 1
        
        return result.astype(int)
    
    def find_most_likely_source(self, anomalies_results):
        anomaly_exists = False
        for i in range(anomalies_results.shape[0]):
            if anomalies_results[i] == 1:
                anomaly_exists = True
                break
        if not anomaly_exists:
            return -1
        
        evidence = {}
        for i in range(self.num_services):
            update_value = 3 # Default to >15 min
            time_since_last_update = time.time() - self.recent_update_times[i][0]
            
            if time_since_last_update <= 1 * 60:
                update_value = 0
            elif time_since_last_update <= 5 * 60:
                update_value = 1
            elif time_since_last_update <= 15 * 60:
                update_value = 2
                
            evidence["U" + str(i)] = update_value
            
        for j in range(self.num_timeseries):
            evidence["A" + str(j)] = anomalies_results[j]
        
        variables = ["S" + str(i) for i in range(self.num_services)]
        
        highest_probability = -1
        most_likely_service_id = -1

        probabilities = self.ve.query(variables, evidence = evidence)
        
        for i in range(self.num_services):
            probability = probabilities["S" + str(i)].values[1]
            if probability > highest_probability:
                highest_probability = probability
                most_likely_service_id = i
            
        return most_likely_service_id
    
    def report_invalid_anomaly_result(self, timeseries_id, anomaly_was_missed, step_size = 0.001):
        delta = step_size
        if not anomaly_was_missed:
            delta *= -1
        self.thresholds[timeseries_id][0] += delta
        
    def report_incorrect_source_estimation(self, anomalies_results, incorrectly_estimated_service_id,
                                           correct_service_id, step_size = 0.001):
        old_cpds = []
        new_cpds = []
        
        for j in range(anomalies_results.shape[0]):
            if anomalies_results[j][0] == 1:
                self.likelihoods[incorrectly_estimated_service_id][j] -= step_size
                self.likelihoods[correct_service_id][j] += step_size
                old_cpds.append(self.network_bayes_model.get_cpds("A" + str(j)))
                new_cpds.append(create_anomaly_node_cpd(self.svc_failure_priors, self.likelihoods[:, j], j))
                
        self.network_bayes_model.remove_cpds(*old_cpds)
        self.network_bayes_model.add_cpds(*new_cpds)
        
        if not self.network_bayes_model.check_model():
            raise Exception("Bayesian model failure")
        
        self.ve = VE(self.network_bayes_model)

## Example

I will explore a simple example of this system below. Imagine a social networking website with a chat service. The chat service only functions when the website functions, but the website offers a variety of features outside of chat that are much more important to the usage numbers. Twitter is a good website to use as a concrete version of this example.

The graph of nodes can be seen below:

<img src="images/example_dag.jpg" />

Now, in code, I can easily create this entire network with just a handful of lines, most of which are just creating dummy matrices:

In [11]:
# test time
example_means = [_get_mean_at_time(t) for t in range(86400)]
example_variances = [_get_variance_at_time(t) for t in range(86400)]

example_model = TimeseriesNetworkModel(
    np.asarray([[1], [1]]),
    np.asarray([
        [0.1275, 0.0650, 0.0120, 0.0001],
        [0.1275, 0.0650, 0.0120, 0.0001]
    ]),
    np.asarray([
        [0.0010],
        [0.0010]
    ]),
    np.asarray([
        [0.8750, 0],
        [0.9500, 0.9750]
    ]),
    np.asarray([0.2, 0.2]),
    [example_means, example_means],
    [example_variances, example_variances]
)

Now I can test this. I will just create three time series points, each with the chat readings going outside their expected range but the website readings exactly at the mean.

In [12]:
example_model.record_slope_reading(0, -0.5)
example_model.record_slope_reading(1, _get_mean_at_time(0))

example_model.record_slope_reading(0, -0.5)
example_model.record_slope_reading(1, _get_mean_at_time(1))

example_model.record_slope_reading(0, -0.5)
example_model.record_slope_reading(1, _get_mean_at_time(2))

And now I'll set the update time of the chat service to be this moment and then immediately run the anomaly detection method. We should expect to see that chat is held responsible for the anomaly.

In [13]:
service_names = ["Chat", "Website"]

example_model.record_service_update(0)

anomalies_results = example_model.check_for_anomalies(3)
most_likely_source_service_id = example_model.find_most_likely_source(anomalies_results)

print("The most likely cause is service", service_names[most_likely_source_service_id])

The most likely cause is service Chat


And it worked!